<a href="https://colab.research.google.com/github/ayaresponsi/Analisis-SMS-Menggunakan-AI-Tools/blob/main/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.0 MB

In [2]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
# Set the API token
api_token = userdata.get('REPLICATE_API_TOKEN')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.2-8b-instruct"
output = Replicate(
  model=model,
  replicate_api_token=api_token,
)

In [4]:
# Load the CSV (adjust the path if needed)
import pandas as pd
df = pd.read_csv('/content/spam.csv', sep='\t', encoding='latin-1', names=['v1', 'v2'])

# Sample the DataFrame to reduce the size of the input data
sampled_df = df.sample(n=1000, random_state=42) # Sample 1000 rows for analysis

# 4. Format sampled messages for Granite
formatted_messages = ""
for i, row in sampled_df.iterrows():
    formatted_messages += f"v1: {row['v1']} | v2: {row['v2']}\n"

# 5. Create the prompt for Granite
prompt = f"""
You are given a sample of the SMS Spam Collection dataset. Each message is labeled as either 'ham' (normal) or 'spam'.

Your tasks:
1. Analyze the content and distribution of spam and ham messages in this sample.
2. Identify patterns, keywords, or linguistic features that differentiate spam from ham based on the sample.
3. Generate analytical results such as common spam words, length differences, frequency trends, etc. from the sample.
4. Share insights that explain the nature of the spam content observed in the sample.
5. Recommend strategies to filter or manage spam effectively based on your analysis of the sample.

Here is the data sample:
{formatted_messages}
"""

# 6. Send the prompt to Granite
response = output.invoke(prompt)

# 7. Print the result
print("Granite Model Response:\n")
print(response)

Granite Model Response:

1. **Content and Distribution Analysis:**

The provided sample consists of 552 SMS messages, with 287 labeled as 'ham' (normal, non-spam) and 265 labeled as 'spam'. This indicates a relatively high proportion of spam messages (48%) compared to normal messages (52%). 

2. **Patterns, Keywords, and Linguistic Features:**

- **Urgency and Prizes:** Spam messages often contain urgent language, promising immediate rewards such as prizes, cash, or free gifts (e.g., "URGENT! Your Mobile No was awarded a £2,000 prize GUARANTEED. Call 09061790125 from landline. Claim 3030. Valid 12hrs only 150ppm").

- **Sexual Innuendo or Explicit Content:** Many spam messages contain explicit or sexually suggestive language, attempting to lure recipients into premium-rate services (e.g., "Hey you can pay. With salary de. Only £...").

- **Miscellaneous Offers:** A variety of unrelated offers are included in spam messages, such as job offers, ringtones, and horoscopes (e.g., "Join the 

In [5]:
import random

# Set seed agar hasil sampling selalu sama
random.seed(42)

# 1. Baca file sebagai teks
with open('/content/spam.csv', 'r', encoding='latin-1') as file:
    lines = file.readlines()

# 2. Ambil 150 pesan secara acak dari seluruh baris
sampled_lines = random.sample(lines, 150)

# 3. Format setiap pesan dengan panjang maksimal 150 karakter
max_len = 150
formatted_messages = ""
for i, line in enumerate(sampled_lines):
    message = line.strip()
    if message:
        truncated = message[:max_len] + ("..." if len(message) > max_len else "")
        formatted_messages += f"Message {i+1}: {truncated}\n"

# 4. Buat prompt untuk dikirim ke Granite
prompt = f"""
You are analyzing SMS messages that may contain spam. Each line below is a truncated message (max 150 chars), no labels provided.

Your tasks:
1. Detect patterns that could indicate spam vs non-spam (keywords, structure, tone).
2. Generate insights from the message content (frequency of promos, length, tone).
3. Provide analytical results based on the sample data.
4. Suggest recommendations to detect or filter spam automatically.

Here is the data:
{formatted_messages}
"""

# 5. Kirim ke Granite
response = output.invoke(prompt)

# 6. Tampilkan hasil dari Granite
print("Granite Model Response:\n")
print(response)


Granite Model Response:

1. Patterns Indicating Spam vs Non-Spam:

   - Spam: Often includes promotional content, unsolicited offers, excessive punctuation, abbreviations, or numbers (like phone numbers or offer codes). They may also have a high frequency of certain keywords such as "free," "win," "offer," "claim," "text now," "stop," etc.

   - Non-Spam: Generally, more personal, less promotional, and written in standard English with proper grammar and punctuation. They often ask questions or express personal thoughts or feelings.

2. Insights from Message Content:

   - Frequency of Promos: There are several messages (e.g., Messages 18, 19, 28, 34, 55, 94, 95, 96, 104, 115, 124, 135) that contain promotions or offers, suggesting a high frequency of promotional content in spam messages.

   - Length: Spam messages tend to be shorter and more concise, often using abbreviations and numbers. Non-spam messages are generally longer and more detailed.

   - Tone: Spam messages often have a 

In [6]:
sampled_lines = random.sample(lines, 500)

max_len = 150
formatted_messages = ""
for i, line in enumerate(sampled_lines):
    message = line.strip()
    if message:
        truncated = message[:max_len] + ("..." if len(message) > max_len else "")
        formatted_messages += f"Message {i+1}: {truncated}\n"

prompt = f"""
You are analyzing SMS messages that may contain spam. Each line below is a truncated message (max 150 chars), no labels provided.

Your tasks:
1. Detect patterns that could indicate spam vs non-spam (keywords, structure, tone).
2. Generate insights from the message content (frequency of promos, length, tone).
3. Provide analytical results based on the sample data.
4. Suggest recommendations to detect or filter spam automatically.

Here is the data:
{formatted_messages}
"""

response = output.invoke(prompt)
print("Granite Model Response:\n")
print(response)

Granite Model Response:

1. **Spam vs Non-Spam Patterns**:
   - **Spam Indicators**:
     - Use of premium rate numbers (e.g., 0906, 0871) for prize claims, horoscopes, or dating services.
     - Vague or generic greetings (e.g., "Dear 0776xxxxxxx", "Hi baby").
     - Promises of prizes, discounts, or special offers without a clear context or personalization.
     - Use of excessive punctuation (!;;), abbreviations, or non-standard spellings (e.g., "wun", "dun").
     - Lack of personal context or coherence in the message.
     - Use of urgent or threatening language (e.g., "URGENT", "YOU'RE LATE", "YOU'RE BEING CONTACTED").
   - **Non-Spam Indicators**:
     - Personalized greetings or context (e.g., mentioning specific events, places, or shared history).
     - Clear, coherent, and grammatically correct language.
     - Use of standard English spelling and punctuation.
     - Messages related to everyday activities, such as arranging meetups, discussing work, or sharing personal upda